# Impossible Travel (Privileged Account)

**Privileged sign-ins from different geolocations within an unrealistic time window.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **geo change**, **device change**, **risk score**, **conditional access**, **token issuance**.


## Investigation Queries

Run platform queries to confirm two sign-ins with different geos < 1 hour. Then check device IDs and apps.


In [ ]:
// KQL
SigninLogs
| where TimeGenerated > ago(24h)
| where UserPrincipalName == '<USER>'
| project TimeGenerated, UserPrincipalName, IPAddress, Location, AppDisplayName, ResultType, DeviceDetail
| order by TimeGenerated asc


In [ ]:
# Splunk SPL
index=auth user=<USER> sourcetype=azuread:signin
| table _time user src_ip country app result
| sort 0 _time


In [ ]:
# Elastic (KQL/EQL)
event.category:authentication AND user.name:"<USER>"


## Containment & Response

If suspicious: revoke sessions, reset credentials, and enforce phishing-resistant MFA for privileged roles.


In [ ]:
# PowerShell
## Revoke refresh tokens (AzureAD)
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>

## Disable account (short-term)
Set-AzureADUser -ObjectId <USER_OBJECT_ID> -AccountEnabled $false


In [ ]:
# PowerShell
## Conditional Access / risk response is tenant-specific.
## Consider requiring compliant device + FIDO2 for admins.
